# Download Chicago's Reported Crime Data

## Download and install Spark

In [1]:
#List the files on the virtual machine
!ls

sample_data


In [2]:
#Download Apache Spark 
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:13 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [1,093 kB]
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.

## Setup environment

In [3]:
#Setup and Initiate Apache Spark Environment

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 
spark

## Downloading and preprocessing Chicago's Reported Crime Data

In [4]:
#Download the data

!wget https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD

--2020-09-17 13:58:01--  https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD
Resolving data.cityofchicago.org (data.cityofchicago.org)... 52.206.140.205, 52.206.140.199, 52.206.68.26
Connecting to data.cityofchicago.org (data.cityofchicago.org)|52.206.140.205|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘rows.csv?accessType=DOWNLOAD’

rows.csv?accessType     [         <=>        ]   1.58G  3.14MB/s    in 8m 52s  

2020-09-17 14:06:54 (3.04 MB/s) - ‘rows.csv?accessType=DOWNLOAD’ saved [1696724428]



In [7]:
!ls

crimes.csv   spark-2.3.1-bin-hadoop2.7
sample_data  spark-2.3.1-bin-hadoop2.7.tgz


In [6]:
#Rename the file

!mv rows.csv\?accessType\=DOWNLOAD crimes.csv

In [8]:
!ls

crimes.csv   spark-2.3.1-bin-hadoop2.7
sample_data  spark-2.3.1-bin-hadoop2.7.tgz


In [11]:
#Import the data as dataframe 
from pyspark.sql.functions import to_timestamp,col,lit
rc = spark.read.csv('crimes.csv',header=True).withColumn('Date',to_timestamp(col('Date'),'MM/dd/yyyy hh:mm:ss a')).filter(col('Date') <= lit('2018-11-11'))
rc.show(5)

+--------+-----------+-------------------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|      ID|Case Number|               Date|               Block|IUCR|       Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|Latitude|Longitude|Location|
+--------+-----------+-------------------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|11034701|   JA366925|2001-01-01 11:00:00|     016XX E 86TH PL|1153| DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|           RESIDENCE| false|   false|0412|     004|   8|            45|      11| 

In [17]:
#Drop some columns
rc=rc.drop('X Coordinate','Y Coordinate','Latitude','Longitude','Location')
rc.show(5)


+--------+-----------+-------------------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+----+--------------------+
|      ID|Case Number|               Date|               Block|IUCR|       Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|Year|          Updated On|
+--------+-----------+-------------------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+----+--------------------+
|11034701|   JA366925|2001-01-01 11:00:00|     016XX E 86TH PL|1153| DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|           RESIDENCE| false|   false|0412|     004|   8|            45|      11|2001|08/05/2017 03:50:...|
|11227287|   JB147188|2017-10-08 03:00:00|  092XX S RACINE AVE|0281|CRIM SEXUAL ASSAULT|      NON-AGGRAVATED|           RESIDENCE| false

In [18]:
!ls

crimes.csv   spark-2.3.1-bin-hadoop2.7	    spark-warehouse
sample_data  spark-2.3.1-bin-hadoop2.7.tgz


In [30]:
#View the contents of single column
rc.select('Description').show(10)

+--------------------+
|         Description|
+--------------------+
|FINANCIAL IDENTIT...|
|      NON-AGGRAVATED|
|      UNLAWFUL ENTRY|
|           OVER $500|
|      NON-AGGRAVATED|
|CRIM SEX ABUSE BY...|
|AGG SEX ASSLT OF ...|
|           OVER $500|
|FINANCIAL IDENTIT...|
|         TO PROPERTY|
+--------------------+
only showing top 10 rows



In [43]:
#Filter the dataframe on specific feature value
rc[rc.Arrest.isin("true")].show(5)

+--------+-----------+-------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+----+--------------------+
|      ID|Case Number|               Date|               Block|IUCR|        Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|Year|          Updated On|
+--------+-----------+-------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+----+--------------------+
|11042582|   JA377037|2011-01-01 00:01:00|054XX S CALIFORNI...|1754|OFFENSE INVOLVING...|AGG SEX ASSLT OF ...|           APARTMENT|  true|    true|0923|     009|  14|            63|      02|2011|08/13/2017 03:50:...|
|10581023|   HZ329792|2014-09-01 08:00:00|     0000X E LAKE ST|1140|  DECEPTIVE PRACTICE|        EMBEZZLEMENT|               OTHER| 

In [44]:
rc.take(3)

[Row(ID='11034701', Case Number='JA366925', Date=datetime.datetime(2001, 1, 1, 11, 0), Block='016XX E 86TH PL', IUCR='1153', Primary Type='DECEPTIVE PRACTICE', Description='FINANCIAL IDENTITY THEFT OVER $ 300', Location Description='RESIDENCE', Arrest='false', Domestic='false', Beat='0412', District='004', Ward='8', Community Area='45', FBI Code='11', Year='2001', Updated On='08/05/2017 03:50:08 PM'),
 Row(ID='11227287', Case Number='JB147188', Date=datetime.datetime(2017, 10, 8, 3, 0), Block='092XX S RACINE AVE', IUCR='0281', Primary Type='CRIM SEXUAL ASSAULT', Description='NON-AGGRAVATED', Location Description='RESIDENCE', Arrest='false', Domestic='false', Beat='2222', District='022', Ward='21', Community Area='73', FBI Code='02', Year='2017', Updated On='02/11/2018 03:57:41 PM'),
 Row(ID='11227583', Case Number='JB147595', Date=datetime.datetime(2017, 3, 28, 14, 0), Block='026XX W 79TH ST', IUCR='0620', Primary Type='BURGLARY', Description='UNLAWFUL ENTRY', Location Description='OTH

In [45]:
rc.show(3)

+--------+-----------+-------------------+------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+----+--------------------+
|      ID|Case Number|               Date|             Block|IUCR|       Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|Year|          Updated On|
+--------+-----------+-------------------+------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+----+--------------------+
|11034701|   JA366925|2001-01-01 11:00:00|   016XX E 86TH PL|1153| DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|           RESIDENCE| false|   false|0412|     004|   8|            45|      11|2001|08/05/2017 03:50:...|
|11227287|   JB147188|2017-10-08 03:00:00|092XX S RACINE AVE|0281|CRIM SEXUAL ASSAULT|      NON-AGGRAVATED|           RESIDENCE| false|   false|

In [47]:
rc.limit(5)

DataFrame[ID: string, Case Number: string, Date: timestamp, Block: string, IUCR: string, Primary Type: string, Description: string, Location Description: string, Arrest: string, Domestic: string, Beat: string, District: string, Ward: string, Community Area: string, FBI Code: string, Year: string, Updated On: string]

In [48]:
rc.head(4)

[Row(ID='11034701', Case Number='JA366925', Date=datetime.datetime(2001, 1, 1, 11, 0), Block='016XX E 86TH PL', IUCR='1153', Primary Type='DECEPTIVE PRACTICE', Description='FINANCIAL IDENTITY THEFT OVER $ 300', Location Description='RESIDENCE', Arrest='false', Domestic='false', Beat='0412', District='004', Ward='8', Community Area='45', FBI Code='11', Year='2001', Updated On='08/05/2017 03:50:08 PM'),
 Row(ID='11227287', Case Number='JB147188', Date=datetime.datetime(2017, 10, 8, 3, 0), Block='092XX S RACINE AVE', IUCR='0281', Primary Type='CRIM SEXUAL ASSAULT', Description='NON-AGGRAVATED', Location Description='RESIDENCE', Arrest='false', Domestic='false', Beat='2222', District='022', Ward='21', Community Area='73', FBI Code='02', Year='2017', Updated On='02/11/2018 03:57:41 PM'),
 Row(ID='11227583', Case Number='JB147595', Date=datetime.datetime(2017, 3, 28, 14, 0), Block='026XX W 79TH ST', IUCR='0620', Primary Type='BURGLARY', Description='UNLAWFUL ENTRY', Location Description='OTH

In [49]:
rc.limit(4).collect()

[Row(ID='11034701', Case Number='JA366925', Date=datetime.datetime(2001, 1, 1, 11, 0), Block='016XX E 86TH PL', IUCR='1153', Primary Type='DECEPTIVE PRACTICE', Description='FINANCIAL IDENTITY THEFT OVER $ 300', Location Description='RESIDENCE', Arrest='false', Domestic='false', Beat='0412', District='004', Ward='8', Community Area='45', FBI Code='11', Year='2001', Updated On='08/05/2017 03:50:08 PM'),
 Row(ID='11227287', Case Number='JB147188', Date=datetime.datetime(2017, 10, 8, 3, 0), Block='092XX S RACINE AVE', IUCR='0281', Primary Type='CRIM SEXUAL ASSAULT', Description='NON-AGGRAVATED', Location Description='RESIDENCE', Arrest='false', Domestic='false', Beat='2222', District='022', Ward='21', Community Area='73', FBI Code='02', Year='2017', Updated On='02/11/2018 03:57:41 PM'),
 Row(ID='11227583', Case Number='JB147595', Date=datetime.datetime(2017, 3, 28, 14, 0), Block='026XX W 79TH ST', IUCR='0620', Primary Type='BURGLARY', Description='UNLAWFUL ENTRY', Location Description='OTH

In [50]:
#Check the datatypes
rc.dtypes

[('ID', 'string'),
 ('Case Number', 'string'),
 ('Date', 'timestamp'),
 ('Block', 'string'),
 ('IUCR', 'string'),
 ('Primary Type', 'string'),
 ('Description', 'string'),
 ('Location Description', 'string'),
 ('Arrest', 'string'),
 ('Domestic', 'string'),
 ('Beat', 'string'),
 ('District', 'string'),
 ('Ward', 'string'),
 ('Community Area', 'string'),
 ('FBI Code', 'string'),
 ('Year', 'string'),
 ('Updated On', 'string')]

In [51]:
#Check the datatypes aka Schema
rc.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: string (nullable = true)
 |-- Domestic: string (nullable = true)
 |-- Beat: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Ward: string (nullable = true)
 |-- Community Area: string (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Updated On: string (nullable = true)



In [52]:
#Column Names
rc.columns

['ID',
 'Case Number',
 'Date',
 'Block',
 'IUCR',
 'Primary Type',
 'Description',
 'Location Description',
 'Arrest',
 'Domestic',
 'Beat',
 'District',
 'Ward',
 'Community Area',
 'FBI Code',
 'Year',
 'Updated On']

In [54]:
rc.select('Community Area','District').show(3)

+--------------+--------+
|Community Area|District|
+--------------+--------+
|            45|     004|
|            73|     022|
|            70|     008|
+--------------+--------+
only showing top 3 rows



In [59]:
#Add New column

rc=rc.withColumn('Double',rc['ward']*2) 
rc.show(3)

+--------+-----------+-------------------+------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+----+--------------------+------+
|      ID|Case Number|               Date|             Block|IUCR|       Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|Year|          Updated On|Double|
+--------+-----------+-------------------+------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+----+--------------------+------+
|11034701|   JA366925|2001-01-01 11:00:00|   016XX E 86TH PL|1153| DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|           RESIDENCE| false|   false|0412|     004|   8|            45|      11|2001|08/05/2017 03:50:...|  16.0|
|11227287|   JB147188|2017-10-08 03:00:00|092XX S RACINE AVE|0281|CRIM SEXUAL ASSAULT|      NON-AGGRAVATED|         

In [62]:
#Rename Columns 

rc=rc.withColumnRenamed('Double','ward_2')
rc.show(3)

+--------+-----------+-------------------+------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+----+--------------------+------+
|      ID|Case Number|               Date|             Block|IUCR|       Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|Year|          Updated On|ward_2|
+--------+-----------+-------------------+------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+----+--------------------+------+
|11034701|   JA366925|2001-01-01 11:00:00|   016XX E 86TH PL|1153| DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|           RESIDENCE| false|   false|0412|     004|   8|            45|      11|2001|08/05/2017 03:50:...|  16.0|
|11227287|   JB147188|2017-10-08 03:00:00|092XX S RACINE AVE|0281|CRIM SEXUAL ASSAULT|      NON-AGGRAVATED|         

In [64]:
#Drop the column

rc=rc.drop('ward_2')
rc.show(3)

+--------+-----------+-------------------+------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+----+--------------------+
|      ID|Case Number|               Date|             Block|IUCR|       Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|Year|          Updated On|
+--------+-----------+-------------------+------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+----+--------------------+
|11034701|   JA366925|2001-01-01 11:00:00|   016XX E 86TH PL|1153| DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|           RESIDENCE| false|   false|0412|     004|   8|            45|      11|2001|08/05/2017 03:50:...|
|11227287|   JB147188|2017-10-08 03:00:00|092XX S RACINE AVE|0281|CRIM SEXUAL ASSAULT|      NON-AGGRAVATED|           RESIDENCE| false|   false|

In [65]:
rc_new=rc.groupBy('Primary Type')

In [71]:
from pyspark.sql.functions import lit
rc.withColumn('One',lit(1)).show(5)

+--------+-----------+-------------------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+----+--------------------+---+
|      ID|Case Number|               Date|               Block|IUCR|       Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|Year|          Updated On|One|
+--------+-----------+-------------------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+----+--------------------+---+
|11034701|   JA366925|2001-01-01 11:00:00|     016XX E 86TH PL|1153| DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|           RESIDENCE| false|   false|0412|     004|   8|            45|      11|2001|08/05/2017 03:50:...|  1|
|11227287|   JB147188|2017-10-08 03:00:00|  092XX S RACINE AVE|0281|CRIM SEXUAL ASSAULT|      NON-AGGRAVATED|           

In [73]:
#Filters
rc.filter(rc.District=='004').show(3)

+--------+-----------+-------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+----+--------------------+
|      ID|Case Number|               Date|               Block|IUCR|        Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|Year|          Updated On|
+--------+-----------+-------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+----+--------------------+
|11034701|   JA366925|2001-01-01 11:00:00|     016XX E 86TH PL|1153|  DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|           RESIDENCE| false|   false|0412|     004|   8|            45|      11|2001|08/05/2017 03:50:...|
|11227247|   JB147078|2012-01-01 09:00:00|105XX S INDIANAPO...|1153|  DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|           RESIDENCE| 

In [76]:
#unique values

rc.select('District').distinct().show()

+--------+
|District|
+--------+
|     009|
|     012|
|     024|
|    null|
|     031|
|     015|
|     006|
|     019|
|     020|
|     011|
|     025|
|     003|
|     005|
|     016|
|     018|
|     008|
|     022|
|     001|
|     014|
|     010|
+--------+
only showing top 20 rows



In [78]:
#Sorting
rc.orderBy('ID',ascending=False).show(3)

+-------+-----------+-------------------+-------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+----+--------------------+
|     ID|Case Number|               Date|              Block|IUCR|Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|Year|          Updated On|
+-------+-----------+-------------------+-------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+----+--------------------+
|9999999|   HY189683|2015-03-18 17:08:00|043XX N CENTRAL AVE|0560|     ASSAULT|              SIMPLE|          RESTAURANT|  true|   false|1624|     016|  38|            15|     08A|2015|02/10/2018 03:50:...|
|9999998|   HY188874|2015-03-18 04:00:00|036XX W BELMONT AVE|0460|     BATTERY|              SIMPLE|           RESIDENCE| false|   false|2523|     025|  35|            21| 

In [79]:
one_day = spark.read.csv('crimes.csv',header=True).withColumn('Date',to_timestamp(col('Date'),'MM/dd/yyyy hh:mm:ss a')).filter(col('Date') == lit('2018-11-12'))
one_day.show(5)

+--------+-----------+-------------------+-------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|               Date|              Block|IUCR|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+-------------------+-------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|11505149|   JB513151|2018-11-12 00:00:00| 003XX S WHIPPLE ST|0810|             THEFT|           OVER $500|              STREET| false| 

In [80]:
one_day.count()

3

In [87]:
one_day=one_day.drop('X Coordinate','Y Coordinate','Latitude','Longitude','Location')
one_day.show(5)

+--------+-----------+-------------------+-------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+----+--------------------+
|      ID|Case Number|               Date|              Block|IUCR|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|Year|          Updated On|
+--------+-----------+-------------------+-------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+----+--------------------+
|11505149|   JB513151|2018-11-12 00:00:00| 003XX S WHIPPLE ST|0810|             THEFT|           OVER $500|              STREET| false|   false|1124|     011|  28|            27|      06|2018|11/19/2018 04:22:...|
|11516594|   JB528186|2018-11-12 00:00:00|049XX S PRAIRIE AVE|2826|     OTHER OFFENSE|HARASSMENT BY ELE...|               OTHER| false|   false|

In [81]:
#Count of rows
rc.count()

6753364

In [91]:
#Union operations(Make sure number of columns are equal)
rc=rc.union(one_day).orderBy('Date',ascending=False)

In [92]:
#Sorting
rc.orderBy('Date',ascending=False).show(3)

+--------+-----------+-------------------+-------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+----+--------------------+
|      ID|Case Number|               Date|              Block|IUCR|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|Year|          Updated On|
+--------+-----------+-------------------+-------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+----+--------------------+
|11540042|   JB559262|2018-11-12 00:00:00|010XX N DEARBORN ST|1140|DECEPTIVE PRACTICE|        EMBEZZLEMENT|   CONVENIENCE STORE|  true|   false|1824|     018|   2|             8|      12|2018|03/16/2019 04:01:...|
|11505149|   JB513151|2018-11-12 00:00:00| 003XX S WHIPPLE ST|0810|             THEFT|           OVER $500|              STREET| false|   false|

In [102]:
#Grouping[Aggregation] and Sorting
rc.groupBy('Primary Type').count().orderBy('count',ascending=False).show(10)

+-------------------+-------+
|       Primary Type|  count|
+-------------------+-------+
|              THEFT|1418436|
|            BATTERY|1232226|
|    CRIMINAL DAMAGE| 771499|
|          NARCOTICS| 711727|
|      OTHER OFFENSE| 418851|
|            ASSAULT| 418508|
|           BURGLARY| 388031|
|MOTOR VEHICLE THEFT| 314132|
| DECEPTIVE PRACTICE| 265933|
|            ROBBERY| 255599|
+-------------------+-------+
only showing top 10 rows



In [105]:
#% of total crimes reported in arrest
(rc[rc['Arrest']=='true'].count()/rc.count())*100

27.754644461051797

In [106]:
#Top 3 locations for reported crimes

rc.groupBy('Location Description').count().orderBy('count',ascending=False).show(3)

+--------------------+-------+
|Location Description|  count|
+--------------------+-------+
|              STREET|1770574|
|           RESIDENCE|1144860|
|           APARTMENT| 698286|
+--------------------+-------+
only showing top 3 rows

